In [4]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Literal, Annotated, List, Union
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from IPython.display import Image, display
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from utils import format_messages


In [5]:
# Load environment variables
load_dotenv()
LLM_KEY = os.getenv("Google_API_KEY")

#tools 

@tool
def calculator(
    operation: Literal["add","subtract","multiply","divide"],
    a: Union[int, float],
    b: Union[int, float],
) -> Union[int, float]:
    """Define a two-input calculator tool.

    Arg:
        operation (str): The operation to perform ('add', 'subtract', 'multiply', 'divide').
        a (float or int): The first number.
        b (float or int): The second number.
        
    Returns:
        result (float or int): the result of the operation
    Example
        Divide: result   = a / b
        Subtract: result = a - b
    """
    if operation == 'divide' and b == 0:
        return {"error": "Division by zero is not allowed."}

    # Perform calculation
    if operation == 'add':
        result = a + b
    elif operation == 'subtract':
        result = a - b
    elif operation == 'multiply':
        result = a * b
    elif operation == 'divide':
        result = a / b
    else: 
        result = "unknown operation"
    return result
    
# Agent Initialization

# Pull the standard ReAct prompt
prompt = """You are a helpful assistant that can perform basic arithmetic operations like addition, subtraction, multiplication, and division. Use the provided tool to perform calculations when necessary."""
model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0)

tools = [calculator]

agent = create_react_agent(model, tools, prompt = prompt).with_config({"recursion_limit": 10})


C:\Users\acer\AppData\Local\Temp\ipykernel_6056\2136865482.py:50: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, tools, prompt = prompt).with_config({"recursion_limit": 10})


In [6]:
type(agent)

langgraph.graph.state.CompiledStateGraph

In [7]:
result1 = agent.invoke({
    "messages" : [{
        "role": "user",
        "content": "What is 3.1 * 4.2?",
    }],
})
format_messages(result1["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ What is 3.1 * 4.2?                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: calculator                                                                                        │
│    Args: {                                                                                                      │
│   "operation": "multiply",                                                                                      │
│   "a": 3.1,                                                                                                     │
│   "b": 4.2                                                                                                      │
│ }                                                                                                               │
│    ID: e1e80533-4a4a-48c1-a62f-fd00e2a216fd                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ 13.020000000000001                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ 3.1 * 4.2 = 13.02                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯